# The AD-AS curve and supply shocks

Imports and set magics:

In [357]:
import numpy as np
from scipy import optimize
import scipy.optimize as opt
import sympy as sm
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider
#from IPython.display import display

plt.rcParams.update({"axes.grid":True,"grid.color":"black","grid.alpha":"0.25","grid.linestyle":"--"})
plt.rcParams.update({'font.size': 14})

# autoreload modules when code is run
%load_ext autoreload
%autoreload 2

# local modules
import modelproject

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Model description of the AD-AS model 


**In a closed economy on the short run**

The AD-AS model is a model from macroeconomics that describes the relationship between aggregate demand (AD) and aggregate supply (AS) in the short run. The model can be used to analyze how changes in government spending, taxes and monetary policy affect the output, prices and employment.

The model below is inspired from the textbook "Introducing Advanced Macroeconomics - Growth and Business Cycles" by Peter Birch Sørensen og Hans Jørgen Whitta-Jacobsen. 

Following five equations describe a closed economy:

**1) The Fischer Equation**:

$$
r = i-\pi^{e}_{+1}
$$

**2) The general market equilibrium**

$$
y-\bar{y} = \alpha_1 (g-\bar{g}) - \alpha_2(r-\bar{r})
$$

**3) The Taylor Rule**

$$
i = \bar{r} + \pi^{e}_{+1} + h(\pi - \pi^*) + b(y- \bar{y})
$$

**4) The SRAS curve** which is derived from the Expectations-Augmented Phillips Curve

$$
\pi = \pi^e + \gamma (y-\bar{y}) + s
$$


**5) Assuming static inflation expectations in the economy**

$$
\pi^e = \pi_{-1}
$$




The **AD-curve** is given by a combination of equation 1), 2) and 3):

$$
y-\bar{y} = z-\alpha(\pi - \pi^*)
$$
where
$$
z = \frac{\alpha_1}{1+\alpha_2b}(g-\bar{g}), \alpha = \frac{\alpha_2 h}{1+\alpha_2b}
$$

The **AS-curve** is given by a combination of equation 4) and 5):

$$
\pi = \pi_{-1} + \gamma(y-\bar{y}) + s

Explanation of the two curves: The **AD curve** represents the relationship between output, Y, and aggregate demand, AD, while the **AS curve** represents the relationship between output, Y, and aggregate supply, AS. 

**Alpha** is a function of the sensitivity of real interest rates,**h** to changes in the output gap and the slope of the Phillips curve, **b**.

**Gamma** is the slope of the AS-curve and **s** represents supply shocks.


Equilibrium: The equilibrium condition for the AD-AS-model is where the two curves intersect.




## Analytical solution

I start by defing the AD- and AS-curve using the equations above:

In [358]:
#Defining the symbols using sympy
y, pi, g, b, alpha1, alpha2, h, s, y_bar, pi_star = sm.symbols('y pi g b alpha1 alpha2 h s y_bar pi_star')

#Defining the AD-curve
z = alpha1/(1 + alpha2*b)*(g - y_bar)
alpha = alpha2*h/(1 + alpha2*b)
AD = y - y_bar - alpha*(pi - pi_star) - z

#Defining the AS-curve
gamma = sm.symbols('gamma')
AS = pi - pi_star + gamma*(y - y_bar) + s

I then derive the steady state equation. This is done by setting AD and AS equal to each other and then solving for y:

In [359]:
#Steady state equation
steady_state_eq = sm.solve(sm.Eq(AD, AS), y)[0]


I now use lambdify to turn it into a Python function:

In [360]:
#Using lambdify to define the SS function
#ss_func takes in the parameters of the model and returns the value of y at SS

ss_func = sm.lambdify((pi, g, b, alpha1, alpha2, h, s, y_bar, pi_star, gamma), steady_state_eq)


I now want to study the AD-AS model when a supply shock hits the economy. I use ipywidgets to illustrate how a supply shock, s, affects the AD- and AS-curve. In period 0 the economy is in its long-term equilibrium. However, in period 1 the user can make a temporary, negative or positive supply shock in the range -5 to +5. Remember: A supply shock is negative when s > 0 and positive when s > 0.  occurs, but disappears again from period 2 onwards.



In [361]:
#Defining functions for the AD and AS curves
def ad(y, pi, pi_star, alpha, alpha_1, alpha_2, b, g, g_bar, h):
    z = alpha_1 / (1 + alpha_2 * b) * (g - g_bar)
    return y - y_bar - z + alpha * (pi - pi_star)

def as_curve(y, pi, pi_1, gamma, s):
    return pi - pi_1 - gamma * (y - y_bar) - s

#Defining the parameter values
y_bar = 100
pi_star = 2
alpha_1 = 1
alpha_2 = 1
b = 1
g = 1
g_bar = 1
h = 1
alpha = alpha_2 * h / (1 + alpha_2 * b)
gamma = 0.5

#Defining a function to plot the AD and AS curves
def plot_ad_as(s):
    #Creating a range of output levels
    y_range = np.linspace(y_bar - 10, y_bar + 10, 100)

    #Deriving the AD and AS curves
    ad_curve = ad(y_range, pi_star, pi_star, alpha, alpha_1, alpha_2, b, g, g_bar, h)
    as_curve_0 = as_curve(y_range, pi_star, pi_star, gamma, 0)
    as_curve_1 = as_curve(y_range, pi_star, pi_star, gamma, s)

    #Plotting
    plt.plot(y_range, ad_curve, label='AD')
    plt.plot(y_range, as_curve_0, label='AS (s=0)', color='grey')
    plt.plot(y_range, as_curve_1, label='AS (s={:.2f})'.format(s), color='red')

    
    plt.xlabel('Output')
    plt.ylabel('Inflation')
    plt.legend()

#Creating the interactive plot with a slider 
interact(plot_ad_as, s=FloatSlider(min=-5, max=5, step=0.1, value=0));


interactive(children=(FloatSlider(value=0.0, description='s', max=5.0, min=-5.0), Output()), _dom_classes=('wi…

It is worth noting, that the AD curve is upward sloping and the AS curve is downward sloping in this model. This is due to the assumptions that are made. For example, an increase in the price level leads to an increase in aggregate demand. This can be counterintuitive, because one would think a higher price level leads to a decrease in demand. But in this case, it is due to the effect of higher inflation on the real interest rate, which thereby affects consumption and investments.




## Numerical solution

You can always solve a model numerically. 

Define first the set of parameters you need. 

Then choose one of the optimization algorithms that we have gone through in the lectures based on what you think is most fitting for your model.

Are there any problems with convergence? Does the model converge for all starting values? Make a lot of testing to figure these things out. 

In [362]:
def social_loss_function(params):
    # Unpack parameter values
    y, pi, g, b, alpha1, alpha2, h, s, y_bar, pi_star, gamma = params
    
    # Compute output gap and inflation deviation from target
    output_gap = y - y_bar
    inflation_dev = pi - pi_star
    
    # Compute social loss function
    loss = output_gap**2 + gamma*inflation_dev**2
    
    return loss



In [363]:
def social_loss_function(params):
    # unpack the parameter values
    pi_star, alpha1, alpha2, b, g, g_bar, h, y_bar, gamma = params
    
    # calculate steady state output
    y_ss = ss_func(pi_star, g, b, alpha1, alpha2, h, 0, y_bar, pi_star, gamma)
    
    # define target levels of output and inflation
    y_target = y_bar
    pi_target = pi_star
    
    # calculate actual output and inflation for a given s value
    def output_inflation(s):
        y = ss_func(pi_target, g, b, alpha1, alpha2, h, s, y_bar, pi_star, gamma)
        pi = pi_star + gamma * (y - y_bar) + s
        return y, pi
    
    # define the range of s values
    s_range = np.linspace(-5, 5, 1000)
    
    # calculate the squared deviation of output and inflation from target values for each s value
    deviation = []
    for s in s_range:
        y, pi = output_inflation(s)
        deviation.append((y - y_target)**2 + (pi - pi_target)**2)
    
    # calculate the social loss as the sum of squared deviations
    social_loss = np.sum(deviation)
    
    return social_loss


In [364]:
import scipy.optimize as optimize

# define the initial guess for parameter values
params_guess = [pi_star, alpha_1, alpha_2, b, g, g_bar, h, y_bar, gamma]

# perform optimization
result = optimize.minimize(social_loss_function, params_guess, method='SLSQP')

# extract the optimized parameter values
optimized_params = result.x


In [365]:
# extract the optimized parameter values
pi_star_opt, alpha_1_opt, alpha_2_opt, b_opt, g_opt, g_bar_opt, h_opt, y_bar_opt, gamma_opt = optimized_params

# plot the AD and AS curves with the optimized parameter values
plot_ad_as(0, pi_star_opt, alpha_1_opt, alpha_2_opt, b_opt, g_opt, g_bar_opt, h_opt, y_bar_opt, gamma_opt)

# calculate and print the steady state output level with the optimized parameter values
y_ss_opt = ss_func(pi_star_opt, g_opt, b_opt, alpha_1_opt, alpha_2_opt, h_opt, 0, y_bar_opt, pi_star_opt, gamma_opt)
print('Steady state output with optimized parameter values: {:.2f}'.format(y_ss_opt))


TypeError: plot_ad_as() takes 1 positional argument but 10 were given

# Further analysis

Make detailed vizualizations of how your model changes with parameter values. 

Try to make an extension of the model. 

# Conclusion

Add concise conclusion. 